Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Breno de Castro Pimenta"
COLLABORATORS = ""

---

# Lista 09 - Comparando Regressões

# Exercício 01:

Analise o desempenho do kNN e de uma Regressão Linear Regularizada para **pelo menos um** dos conjuntos de dados disponível na [seção de regressão linear múltipla](http://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/frames/frame.html) da página do *Livro Understandable Statistics* de Charles Brase e Corrinne Brase. Para o conjunto de dados que escolheu, execute a regressão linear múltipla para explicar o fator $X1$ dos dados ([ver descrição de um dos conjuntos](http://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/frames/frame.html)) a partir dos outros fatores. 

Para a questão, faça as seguintes tarefas:

* Realize treino, validação e teste
* Compare as métricas no teste
* Diferente da lista anterior, reporte o erro quadrado médio no conjunto de teste.

In [2]:
# YOUR CODE HERE
from scipy import stats as ss

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams['figure.figsize']  = (18, 10)
plt.rcParams['axes.labelsize']  = 20
plt.rcParams['axes.titlesize']  = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['lines.linewidth'] = 4

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import explained_variance_score, r2_score, mean_squared_error, median_absolute_error
from sklearn.feature_selection import RFE

### Objetivo: Neste TP tentaremos prever o preço em dólares do barril de petróleo

In [4]:
df = pd.read_csv('oil_price.csv', sep=';')
df.head(12)

CRUDE   INTEREST  FOREIGN         DJIA     GNP  PURCHASE    CONSUMER
0   10.900000   7.610000     31.0   974.900024  1718.0     1.757  234.399994
1   12.000000   7.420000     35.0   894.599976  1918.0     1.649  263.799988
2   12.500000   8.410000     42.0   820.200012  2164.0     1.532  308.299988
3   17.700001   9.439999     54.0   844.400024  2418.0     1.380  347.500000
4   28.100000  11.460000     83.0   891.400024  2732.0     1.215  349.399994
5   35.599998  13.910000    109.0   932.900024  3053.0     1.098  366.600006
6   31.799999  13.000000    125.0   884.400024  3166.0     1.035  381.100006
7   29.000000  11.110000    137.0  1190.300049  3406.0     1.000  430.399994
8   28.600000  12.440000    165.0  1178.500000  3772.0     0.961  511.799988
9   26.799999  10.620000    185.0  1328.199951  4015.0     0.928  592.400024
10  14.600000   7.680000    209.0  1792.800049  4240.0     0.913  646.099976
11  17.900000   8.380000    244.0  2276.000000  4527.0     0.880  685.500000

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 7 columns):
CRUDE       12 non-null float64
INTEREST    12 non-null float64
FOREIGN     12 non-null float64
DJIA        12 non-null float64
GNP         12 non-null float64
PURCHASE    12 non-null float64
CONSUMER    12 non-null float64
dtypes: float64(7)
memory usage: 752.0 bytes


Vemos um número pequeno de dados, no entanto não há valores faltando e todos os dados são numéricos.

### Separando treino e teste

* Separando o estimador:

In [6]:
X = df.copy()
X.drop('CRUDE', axis=1, inplace=True)
y = df['CRUDE']
X.head(5)

INTEREST  FOREIGN        DJIA     GNP  PURCHASE    CONSUMER
0   7.610000     31.0  974.900024  1718.0     1.757  234.399994
1   7.420000     35.0  894.599976  1918.0     1.649  263.799988
2   8.410000     42.0  820.200012  2164.0     1.532  308.299988
3   9.439999     54.0  844.400024  2418.0     1.380  347.500000
4  11.460000     83.0  891.400024  2732.0     1.215  349.399994

* Separando o dataset em treino e teste, 80/20:

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
print(X_test.shape)

(9, 6)
(3, 6)


### Normalizando:

* Z-normalizando todas as colunas numéricas, no caso deste dataset todas as colunas dos atributos:

In [8]:
mean_train = X_train.mean()
std_train = X_train.std(ddof=1)

Z_train = X_train.copy()
Z_train = X_train - mean_train
Z_train = Z_train / std_train

Z_test = X_test.copy()
Z_test = X_test - mean_train
Z_test = Z_test / std_train

# KNN

### Validação Cruzada:

In [9]:
model = KNeighborsRegressor()

#Gera a Grid
grid_search = GridSearchCV(model,
                           param_grid={"n_neighbors": range(1, 7)},
                           scoring="neg_mean_squared_error",
                           cv=7,
                           iid='deprecated')

#Realiza a busca com os valores do treino:
grid_search.fit(Z_train, y_train)
best_n_neighbors = grid_search.best_estimator_.n_neighbors
print("Melhor número de vizinhos após substituir todos os valores na Grid: ", best_n_neighbors)

Melhor número de vizinhos após substituir todos os valores na Grid:  1


### Treino:

In [10]:
model = KNeighborsRegressor(n_neighbors=best_n_neighbors)
model.fit(Z_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=1, p=2,
          weights='uniform')

### Teste:

In [11]:
previsao_knn = model.predict(Z_test)

In [12]:
#Gerando dados da qualidade do modelo
knn_variancia_err = explained_variance_score(y_true=y_test, y_pred=previsao_knn)
knn_r_quadrado_err = r2_score(y_true=y_test, y_pred=previsao_knn)
knn_erro_medio_quadrado = mean_squared_error(y_true=y_test, y_pred=previsao_knn)
knn_erro_medio_abs = median_absolute_error(y_true=y_test, y_pred=previsao_knn)

score_knn = [knn_variancia_err, knn_r_quadrado_err, knn_erro_medio_abs, knn_erro_medio_quadrado]

# Regressão Linear

### Validação Cruzada e Treino:

Não há hiper parâmetros para tunar utilizando regressão linear, mas para aprimorar o resultado usaremos uma validação cruzada para gerar um peso para as features utillizando validação cruzada, gerando dois modelos lineares, um com validação cruzada e outro não: 

In [13]:
# Cria-se um esquema de validação cruzada para passar para o GridSeach
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

# Range dos hiper parâmetros que serão tunados
hyper_params = [{'n_features_to_select': list(range(1, 6))}]


# step-3: perform grid search
# 3.1 specify model
linear_model = LinearRegression()
linear_model.fit(Z_train, y_train)
rfe = RFE(linear_model)             

# Cria-se um novo modelo com os novos pesos
linear_model_cv = GridSearchCV(estimator = rfe, 
                        param_grid = hyper_params, 
                        scoring= 'r2', 
                        cv = folds,
                        return_train_score=True,
                        iid='deprecated')      

linear_model_cv.fit(Z_train, y_train) 

GridSearchCV(cv=KFold(n_splits=5, random_state=100, shuffle=True),
       error_score='raise-deprecating',
       estimator=RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False),
  n_features_to_select=None, step=1, verbose=0),
       fit_params=None, iid='deprecated', n_jobs=None,
       param_grid=[{'n_features_to_select': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

### Teste:

In [14]:
previsao_lm = linear_model.predict(Z_test)
previsao_lm_cv = linear_model_cv.predict(Z_test)

In [15]:
#Gerando dados da qualidade do modelo linear
lm_variancia_err = explained_variance_score(y_true=y_test, y_pred=previsao_lm)
lm_r_quadrado_err = r2_score(y_true=y_test, y_pred=previsao_lm)
lm_erro_medio_quadrado = mean_squared_error(y_true=y_test, y_pred=previsao_lm)
lm_erro_medio_abs = median_absolute_error(y_true=y_test, y_pred=previsao_lm)

score_lm = [lm_variancia_err, lm_r_quadrado_err, lm_erro_medio_abs, lm_erro_medio_quadrado]

In [16]:
#Gerando dados da qualidade do modelo linear com validação cruzada
lm_cv_variancia_err = explained_variance_score(y_true=y_test, y_pred=previsao_lm_cv)
lm_cv_r_quadrado_err = r2_score(y_true=y_test, y_pred=previsao_lm_cv)
lm_cv_erro_medio_quadrado = mean_squared_error(y_true=y_test, y_pred=previsao_lm_cv)
lm_cv_erro_medio_abs = median_absolute_error(y_true=y_test, y_pred=previsao_lm_cv)

score_lm_cv = [lm_cv_variancia_err, lm_cv_r_quadrado_err, lm_cv_erro_medio_abs, lm_cv_erro_medio_quadrado]

# Comparando Modelos:

In [19]:
comparacao = {}
comparacao['Regressão Linear'] = score_lm
comparacao['Regressão Linear c/ Validação Cruzada'] = score_lm_cv
comparacao['KNN'] = score_knn

df_comparacao = pd.DataFrame.from_dict(comparacao, orient='index', columns=['Variância', 'R^2', 'Erro Médio Absoluto', 'Erro Quadrado Médio'])
df_comparacao

Variância       R^2  \
Regressão Linear                       -2.830509 -4.798074   
Regressão Linear c/ Validação Cruzada   0.936246  0.337178   
KNN                                     0.000000 -0.120215   

                                       Erro Médio Absoluto  \
Regressão Linear                                  7.373555   
Regressão Linear c/ Validação Cruzada             2.617562   
KNN                                               1.600000   

                                       Erro Quadrado Médio  
Regressão Linear                                 51.499794  
Regressão Linear c/ Validação Cruzada             5.887337  
KNN                                               9.950003

Explique e discuta sobre os resultados encontrados no campo abaixo.

Apesar do número baixo de dados os resultados demonstraram clara discrepância.<br>
O pior dos modelos é a regressão linear simples, temos um erro médio absoluto de 7 dólares, onde os preços variam entre 10 e 35 dólares e o erro quadrado médio torna ainda mais evidente o péssimo desempenho deste modelo. <br>
O KNN possui o menor erro absoluto e um erro quadrado médio muito menor que a Regressão Linear simples, mostrando como a escolha do modelo mais apropriado gera uma grade diferença.<br>
No entanto, ao tunar os atributos da Regressão Linear conseguiu-se alcançar um erro quadrático médio melhor do que o KNN, mostrando como que a forma que se encara os dados diante do modelo é um aspecto essencial da ciência dos dados.